In [8]:
import torch
from torch.autograd import Variable

dtype = torch.FloatTensor

N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor 생성, Variable로 감쌈
# requires_grad = False로 설정하여 역전파 중에 이 Variable은 변화도를 계산할 필요가 없음을 나타냄
x = Variable(torch.randn(N, D_in).type(dtype), requires_grad=False)
y = Variable(torch.randn(N, D_out).type(dtype), requires_grad=False)

# 가중치를 저장하기 위해 무작위 값을 갖는 Tensor 생성, Variable로 감쌈
# requires_grad = True로 설정하여 역전파 중에 이 Variable들에 대한 변화도를 계산할 필요가 없음을 나타냄
w1 = Variable(torch.randn(D_in, H).type(dtype), requires_grad=True)
w2 = Variable(torch.randn(H, D_out).type(dtype), requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: Variable 연산을 사용하여 y 값 예측
        # Tensor를 사용한 순전파 단계와 완전히 동일하지만, 역전파 단계를 별도로 구현하지 않기 위해
        # 중간 값들에 대한 참조를 갖고 있을 필요가 없다!
        # 따라서 h, h_relu의 계산 과정까지 한번에 y_pred 안으로!
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # Variable 연산을 사용하여 손실을 계산하고 출력
    # loss는 (1,) 모양의 Variable, loss.data는 (1,)모양의 텐서
    # loss.data[0]은 손실(loss의 스칼라 값)
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.data)
    
    # autograd를 사용한 역전파 단계
    # 'requires_grad=True를 갖는 모든 Variable에 대한 손실의 변화도 계산'
    loss.backward()
    
    # 경사하강법을 사용한 가중치 갱신
    w1.data -= learning_rate * w1.grad.data
    w2.data -= learning_rate * w2.grad.data
    
    # 가중치 갱신 후 수동으로 변화도를 0으로 만들기
    w1.grad.data.zero_()
    w2.grad.data.zero_()

0 tensor(24611484.)
1 tensor(18009332.)
2 tensor(16033823.)
3 tensor(16254675.)
4 tensor(17311768.)
5 tensor(18014242.)
6 tensor(17383404.)
7 tensor(15083868.)
8 tensor(11664259.)
9 tensor(8155498.5000)
10 tensor(5315781.5000)
11 tensor(3350163.5000)
12 tensor(2115881.)
13 tensor(1376783.1250)
14 tensor(939721.5625)
15 tensor(677790.2500)
16 tensor(515387.5625)
17 tensor(409725.7500)
18 tensor(337067.5312)
19 tensor(284299.8125)
20 tensor(244002.8125)
21 tensor(211953.9688)
22 tensor(185692.7812)
23 tensor(163695.0781)
24 tensor(144979.3281)
25 tensor(128872.5703)
26 tensor(114889.0938)
27 tensor(102679.0938)
28 tensor(91977.5078)
29 tensor(82552.0391)
30 tensor(74234.5547)
31 tensor(66864.6484)
32 tensor(60326.8594)
33 tensor(54504.1523)
34 tensor(49308.4453)
35 tensor(44667.3672)
36 tensor(40521.4375)
37 tensor(36816.4297)
38 tensor(33486.0547)
39 tensor(30487.5391)
40 tensor(27784.5020)
41 tensor(25345.6445)
42 tensor(23142.3613)
43 tensor(21149.3945)
44 tensor(19342.9375)
45 tensor

457 tensor(4.5811e-05)
458 tensor(4.5208e-05)
459 tensor(4.4188e-05)
460 tensor(4.3573e-05)
461 tensor(4.2916e-05)
462 tensor(4.2254e-05)
463 tensor(4.1632e-05)
464 tensor(4.1262e-05)
465 tensor(4.0796e-05)
466 tensor(4.0408e-05)
467 tensor(3.9784e-05)
468 tensor(3.9219e-05)
469 tensor(3.8754e-05)
470 tensor(3.8154e-05)
471 tensor(3.7674e-05)
472 tensor(3.7332e-05)
473 tensor(3.6870e-05)
474 tensor(3.6434e-05)
475 tensor(3.6019e-05)
476 tensor(3.5639e-05)
477 tensor(3.5149e-05)
478 tensor(3.4623e-05)
479 tensor(3.4239e-05)
480 tensor(3.3803e-05)
481 tensor(3.3487e-05)
482 tensor(3.3060e-05)
483 tensor(3.2654e-05)
484 tensor(3.2386e-05)
485 tensor(3.2043e-05)
486 tensor(3.1609e-05)
487 tensor(3.1269e-05)
488 tensor(3.0887e-05)
489 tensor(3.0492e-05)
490 tensor(3.0072e-05)
491 tensor(2.9787e-05)
492 tensor(2.9397e-05)
493 tensor(2.9040e-05)
494 tensor(2.8710e-05)
495 tensor(2.8354e-05)
496 tensor(2.8092e-05)
497 tensor(2.7786e-05)
498 tensor(2.7495e-05)
499 tensor(2.7104e-05)


In [7]:
import torch
from torch.autograd import Variable

class MyReLU(torch.autograd.Function):
    """
    torch.autograd.Function을 상속받아 사용자 정의 autograd 함수 구현 후,
    Tensor 연산을 하는 순전파와 역전파 단계 구현
    """
    
    @staticmethod
    def forward(ctx, input):
        """
        ctx는 역전파 연산을 위한 정보를 저장하기 위해 사용하는 Context Object
        ctx.save_for_backward method를 사용하여 역전파 단계에서 사용할 어떤 객체도 저장해둘 수 있다.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        출력에 대한 손실의 변화도를 갖는 Tensor를 받고, 입력에 대한 손실의 변화도 계산
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input<0]=0
        return grad_input

dytpe = torch.FloatTensor
N, D_in, H, D_out = 64, 1000, 100, 10

w1 = Variable(torch.randn(D_in, H).type(dtype), requires_grad=True)
w2 = Variable(torch.randn(H, D_out).type(dtype), requires_grad=True)

learning_rate=1e-6
for t in range(500):
    relu= MyReLU.apply
    
    y_pred = relu(x.mm(w1)).mm(w2)
    
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.data)
    
    loss.backward()
    
    w1.data -= learning_rate * w1.grad.data
    w2.data -= learning_rate * w2.grad.data
    
    w1.grad.data.zero_()
    w2.grad.data.zero_()

0 tensor(27122406.)
1 tensor(22019434.)
2 tensor(19119624.)
3 tensor(16333846.)
4 tensor(13123965.)
5 tensor(9799697.)
6 tensor(6902772.)
7 tensor(4687365.5000)
8 tensor(3161708.7500)
9 tensor(2164721.2500)
10 tensor(1528397.2500)
11 tensor(1119835.2500)
12 tensor(852268.1875)
13 tensor(670722.)
14 tensor(542889.2500)
15 tensor(449174.1250)
16 tensor(377841.3750)
17 tensor(321839.7812)
18 tensor(276766.5000)
19 tensor(239778.3906)
20 tensor(208947.4062)
21 tensor(182959.9375)
22 tensor(160863.6875)
23 tensor(141930.4844)
24 tensor(125634.7266)
25 tensor(111526.7891)
26 tensor(99240.9766)
27 tensor(88509.3359)
28 tensor(79103.4609)
29 tensor(70834.8047)
30 tensor(63543.2930)
31 tensor(57098.6172)
32 tensor(51389.5781)
33 tensor(46322.0273)
34 tensor(41814.5391)
35 tensor(37799.2500)
36 tensor(34216.2031)
37 tensor(31011.4219)
38 tensor(28140.8789)
39 tensor(25563.5332)
40 tensor(23247.1445)
41 tensor(21162.4297)
42 tensor(19283.7656)
43 tensor(17588.7617)
44 tensor(16056.8438)
45 tensor

448 tensor(4.5546e-05)
449 tensor(4.4949e-05)
450 tensor(4.4628e-05)
451 tensor(4.3866e-05)
452 tensor(4.3354e-05)
453 tensor(4.2702e-05)
454 tensor(4.2151e-05)
455 tensor(4.1602e-05)
456 tensor(4.0931e-05)
457 tensor(4.0599e-05)
458 tensor(3.9954e-05)
459 tensor(3.9462e-05)
460 tensor(3.8751e-05)
461 tensor(3.8411e-05)
462 tensor(3.7893e-05)
463 tensor(3.7231e-05)
464 tensor(3.6951e-05)
465 tensor(3.6528e-05)
466 tensor(3.6192e-05)
467 tensor(3.5566e-05)
468 tensor(3.5108e-05)
469 tensor(3.4767e-05)
470 tensor(3.4291e-05)
471 tensor(3.3996e-05)
472 tensor(3.3523e-05)
473 tensor(3.3115e-05)
474 tensor(3.2732e-05)
475 tensor(3.2514e-05)
476 tensor(3.2057e-05)
477 tensor(3.1615e-05)
478 tensor(3.1372e-05)
479 tensor(3.1021e-05)
480 tensor(3.0667e-05)
481 tensor(3.0307e-05)
482 tensor(3.0000e-05)
483 tensor(2.9660e-05)
484 tensor(2.9320e-05)
485 tensor(2.9147e-05)
486 tensor(2.8789e-05)
487 tensor(2.8466e-05)
488 tensor(2.8276e-05)
489 tensor(2.7900e-05)
490 tensor(2.7574e-05)
491 tensor(